# The Apache Cassandra coding

## Purpose : Creating queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## Dataset used: <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Process
1. Prepare database of appache cassandra
2. Creating tables for each query and answering questions
3. DROP tables

# 1.Prepare database of appache cassandra

#### Reference
- CREATE KEYSPACE
(https://docs.datastax.com/en/archived/cql/3.3/cql/cql_reference/cqlCreateKeyspace.html)

In [259]:
import cassandra
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity\
        WITH REPLICATION =\
        {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    )
except Exception as e:
    print(e)

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## 2.Creating tables for each query and answering questions

#### Reference
- How to use row_factory and transform to pandas DataFrame from query data.
(https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas)

### Get data and check
- Content of data
- The number of data

In [260]:
import pandas as pd
import csv

In [261]:
file = "event_datafile_new.csv"
df = pd.read_csv(file)
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Rokia TraorÃÂ©,Stefany,F,0,White,274.88608,free,"Lubbock, TX",693,Zen,83
1,Camila,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",555,Abrazame (Version Acustica),40
2,Carl Thomas,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",698,You Ain't Right (Album Version),40
3,N.E.R.D.,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",78,Provider (Remix Radio Edit),79
4,Lil Jon / The East Side Boyz / DJ Flexx,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Aww Skeet Skeet,29


In [262]:
data_cnt = len(df)
print(data_cnt)

6820


### Query1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4
1. Select PRIMARY KEY
2. Create TABLE
3. Execute QUERY

#### 1.Select PRIMARY KEY
- Select 'sessionId' and 'itemInSession' that's why query1 includes it in the WHERE clause.
- Check them NOT NULL and UNIQUE

In [263]:
df.isnull().sum()

artist           0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
sessionId        0
song             0
userId           0
dtype: int64

In [264]:
df_g_s_i = df.groupby(["sessionId", "itemInSession"])

In [265]:
df_g_s_i.count().max()

artist       1
firstName    1
gender       1
lastName     1
length       1
level        1
location     1
song         1
userId       1
dtype: int64

#### 2.Create TABLE

In [266]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [267]:
df_c_list = list(df.columns)
df_c_list

['artist',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'sessionId',
 'song',
 'userId']

In [268]:
query1 = "CREATE TABLE IF NOT EXISTS music_library"
query1 = query1 + "(sessionId int, itemInSession int, artist text, \
    song text, length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [269]:
with open(file, encoding = "UTF-8") as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1 = "INSERT INTO music_library (sessionid, iteminsession, artist, song, length)"
        query1 = query1 + "VALUES(%s, %s, %s, %s, %s)"
        try:
            session.execute(query1, (int(line[df_c_list.index('sessionId')]), int(line[df_c_list.index('itemInSession')]),
                                     line[df_c_list.index('artist')], line[df_c_list.index('song')],float(line[df_c_list.index('length')])))
        except Exception as e:
            print(e)        

#### 3.Execute query

In [270]:
query1 = "SELECT * FROM music_library WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [271]:
pd.DataFrame(list(rows))    

,sessionid,iteminsession,artist,length,song
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


In [272]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

In [273]:
session.row_factory = pandas_factory
session.default_fetch_size = None
#df1 =rows.current_rows

In [274]:
query1 = "SELECT * FROM music_library WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [275]:
df1 =rows._current_rows

In [276]:
df1

,sessionid,iteminsession,artist,length,song
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


### Query2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
1. Select PRIMARY KEY
2. Create TABLE
3. Execute QUERY

#### 1.Select PRIMARY KEY
- Select 'userid' ,'sessionid' for partition key that's why they are in the WHERE clause. Also select 'itemInSession for sorted clause.
- Check them NOT NULL and UNIQUE

In [277]:
df.isnull().sum()

artist           0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
sessionId        0
song             0
userId           0
dtype: int64

In [278]:
df_g_u_s_i = df.groupby(['userId', 'sessionId', 'itemInSession'])

In [279]:
df_g_u_s_i.count().max()

artist       1
firstName    1
gender       1
lastName     1
length       1
level        1
location     1
song         1
dtype: int64

#### 2.Create TABLE

In [280]:
query2 = "CREATE TABLE IF NOT EXISTS user_library"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))\
    WITH CLUSTERING ORDER BY (itemInSession DESC)"
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [281]:
a1 = df_c_list.index('userId')
b1 = df_c_list.index('sessionId')
c1 = df_c_list.index('itemInSession')
d1 = df_c_list.index('artist')
e1 = df_c_list.index('song')
f1 = df_c_list.index('firstName')
g1 = df_c_list.index('lastName')

with open(file, encoding ='UTF-8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO user_library (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query2 = query2 + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query2, (int(line[a1]), int(line[b1]), int(line[c1]), line[d1], line[e1], line[f1], line[g1]))
        except Exception as e:
            print(e)

#### 3.Execute query

In [282]:
query2 = "SELECT * FROM user_library WHERE userid = 10 AND sessionId = 182 ORDER BY iteminsession DESC"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

In [283]:
df2 = rows._current_rows

In [284]:
df2

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,10,182,3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...
1,10,182,2,Sebastien Tellier,Sylvie,Cruz,Kilometer
2,10,182,1,Three Drives,Sylvie,Cruz,Greece 2000
3,10,182,0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On


### Query3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
1. Select PRIMARY KEY
2. Create TABLE
3. Execute QUERY

#### 1.Select PRIMARY KEY
- Select 'song' for partition key that's why they are in the WHERE clause. Also select 'userid' for clustering and uniquely.
- Check them NOT NULL and UNIQUE

In [285]:
df.isnull().sum()

artist           0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
sessionId        0
song             0
userId           0
dtype: int64

In [286]:
df_g_s_u_s_i = df.groupby(['song', 'userId', 'sessionId', 'itemInSession'])

In [287]:
df_g_s_u_s_i.count().max()

artist       1
firstName    1
gender       1
lastName     1
length       1
level        1
location     1
dtype: int64

#### 2.Create TABLE

In [288]:
query3 = "CREATE TABLE IF NOT EXISTS history"
query3 = query3 + "(song text, userId int, sessionId int, itemInSession int, firstName text, lastName text, PRIMARY KEY(song, userId, sessionId, itemInSession))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [293]:
a1 = df_c_list.index('song')
b1 = df_c_list.index('userId')
c1 = df_c_list.index('sessionId')
d1 = df_c_list.index('itemInSession')
e1 = df_c_list.index('firstName')
f1 = df_c_list.index('lastName')

with open(file, encoding ='UTF-8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO history (song, userid, sessionid, iteminsession, firstname, lastname)"
        query3 = query3 + "VALUES(%s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query3, (line[a1], int(line[b1]), int(line[c1]), int(line[d1]), line[e1], line[f1]))
        except Exception as e:
            print(e)

#### 3.Execute query

In [294]:
query3 = "SELECT * FROM history WHERE song ='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

In [295]:
df3 = rows._current_rows

In [296]:
df3

,song,userid,sessionid,iteminsession,firstname,lastname
0,All Hands Against His Own,29,559,50,Jacqueline,Lynch
1,All Hands Against His Own,80,611,25,Tegan,Levine
2,All Hands Against His Own,95,152,31,Sara,Johnson


## 3.Drop tables

In [303]:
query1 = "DROP TABLE IF EXISTS music_library"

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [304]:
query2 = "DROP TABLE IF EXISTS user_library"

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [305]:
query3 = "DROP TABLE IF EXISTS history"

try:
    session.execute(query3)
except Exception as e:
    print(e)

In [ ]:


session.row_factory = pandas_factory
session.default_fetch_size = None

query = "SELECT ..."
rslt = session.execute(query, timeout=None)
df = rslt._current_rows